# Подготовка данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

In [3]:
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Цели исследования

Моя компания купила крупный сервис предоставляющий книги по подписке. Цель - проанализировать базу данных сервиса  В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

## Исследование данных

In [4]:
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [5]:
query = '''SELECT * FROM authors LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [6]:
query = '''SELECT * FROM publishers LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [7]:
query = '''SELECT * FROM ratings LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [8]:
query = '''SELECT * FROM reviews LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Количество книг вышедших после 1 января 2000 года

In [9]:
query = '''SELECT COUNT(DISTINCT(book_id))
FROM books
WHERE CAST(publication_date AS date) > '2000-01-01';'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


После 1 января 2000 года вышло 819 книг

## Количество обзоров и средняя оценка для каждой книги

In [13]:
query = '''SELECT b.title,
COUNT(DISTINCT re.review_id) AS count_review,
AVG(ra.rating)
FROM books AS b
LEFT JOIN reviews AS re ON b.book_id = re.book_id
LEFT JOIN ratings AS ra ON b.book_id = ra.book_id
GROUP BY b.book_id
ORDER BY COUNT(re.review_id) DESC;'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,title,count_review,avg
0,Twilight (Twilight #1),7,3.662500
1,The Hobbit or There and Back Again,6,4.125000
2,The Catcher in the Rye,6,3.825581
3,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
...,...,...,...
995,Essential Tales and Poems,0,4.000000
996,Leonardo's Notebooks,0,4.000000
997,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667


Больше всего обзоров на книгу Twilight#1 - 7

## Издательство, которое выпустило наибольшее число книг толще 50 страниц

In [15]:
query = '''SELECT p.publisher,
COUNT(b.book_id)
FROM books AS b
LEFT JOIN publishers AS p ON b.publisher_id = p.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher
ORDER BY COUNT(b.book_id) DESC;'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,count
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
...,...,...
329,Turtleback,1
330,Atheneum Books for Young Readers: Richard Jack...,1
331,Penguin Signet,1
332,Victor Gollancz,1


Penguin Books - издательство которое выпустило больше всего книг толщиной более 50 страниц

## Автор с самой высокой средней оценкой книг

In [16]:
query = '''SELECT a.author,
ROUND(AVG(ra.rating), 2)
FROM books AS b
LEFT JOIN authors AS a ON b.author_id = a.author_id
LEFT JOIN ratings AS ra ON b.book_id = ra.book_id
GROUP BY a.author
HAVING COUNT(ra.book_id) > 50
ORDER BY AVG(ra.rating) DESC
LIMIT 1;'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author,round
0,J.K. Rowling/Mary GrandPré,4.29


Самая высокая оценка у книг J.K. Rowling с иллюстрациями Mary GrandPré - 4,29

## Среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [17]:
query = '''WITH users_48 AS (
    SELECT username
    FROM ratings
    GROUP BY username
    HAVING COUNT(rating_id) > 48)
    
SELECT AVG(review_count)
FROM (
    SELECT COUNT(re.review_id) AS review_count
    FROM reviews AS re
    JOIN users_48 AS u ON re.username = u.username
    GROUP BY re.username
) AS user_reviews;'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,avg
0,24.0


В среднем пользователи оставившие больше 48 оценок делают по 24 обзора

Выводы:
- После 1 января 2000 года вышло 819 книг;
- Больше всего обзоров на книгу Twilight#1 - 1120, общее количество обзоров 2793;
- Penguin Books - издательство которое выпустило больше всего книг толщиной более 50 страниц;
- Самая высокая оценка у книг J.K. Rowling с иллюстрациями Mary GrandPré - 4,29;
- В среднем пользователи оставившие больше 48 оценок делают по 24 обзора.